# Ejercicio 10

Toma tu número $p$ de la lista publicada para este ejercicio. 

1. Calcula el símbolo de Jacobi $\left(\frac{-11}{p}\right)$. Si sale $1$, usa el algoritmo de Tonelli-Shanks para hallar las soluciones a la congruencia $x^2 \equiv -11 \pmod{p}$.
2. Usa una de esas soluciones para factorizar el ideal principal $(p) = \left(p, n + \sqrt{-11}\right)\left(p, n + \sqrt{-11}\right)$ como producto de dos ideales.
3. Aplica el algoritmo de Cornachia-Smith modificado a $2p$ y $n$ para encontrar una solución a la ecuación diofántica $4p = x^2 + 11y^2$, y úsala para encontrar una factorización de $p$ en a.e. del cuerpo $\mathbb{Q}\left[\sqrt{p}\right]$.
4. ¿Son ideales principales tus ideales $\left(p, n + \sqrt{-11}\right)$ y $\left(p, n + \sqrt{-11}\right)$?